In [3]:
import json
import numpy as np
def load_groundTruth():
    anns = json.load(open("../annotations.json"))
    categories = [c['name'] for c in anns['categories']]
    ids = []
    file_names = []
    imagesDic = {}
    for x in anns['images']:
        path = "../"+x['path']
        file_names.append(path)
        ids.append(x['id'])
        imagesDic[x['id']] = path
    file_names = np.asarray(file_names)
    #restrict ther the number of filenames to 10
    ids = np.asarray(ids)
    boards=[[] for _ in range(len(file_names))]
    for piece in anns['annotations']['corners']:
        idx = np.where(ids == piece['image_id'])[0][0]
        conors = piece['corners']
        boards[idx]= conors#["chessred2k"]
    split_ids = np.asarray(anns['splits']['test']['image_ids']).astype(int)

    intersect = np.isin(ids, split_ids)
    split_ids = np.where(intersect)[0]
    file_names = file_names[split_ids]
    file_names = file_names#[:45]
    #boards = boards[split_ids]
    ids = ids[split_ids]
    splitImages = []
    splitGroundTruth = []
    for id in ids:
        splitImages.append(imagesDic[id])
        splitGroundTruth.append(boards[id])
    #print(f"Number of {partition} images: {len(file_names)}")
    return splitImages ,splitGroundTruth
splitImages ,splitGroundTruth = load_groundTruth()



In [4]:

# Flatten keypoints and normalize
import cv2
image = cv2.imread(splitImages[i])
img_height, img_width = image.shape[:2]
for i in range (len(splitImages)):
    conors = splitGroundTruth[i]
    keypoints = []
    for point in conors.values():
        x, y = point[0] / img_width, point[1] / img_height
        keypoints.extend([x, y, 2])  # 2 = visible and labeled

    # Class ID (0 for "square")
    class_id = 0

    # Save to .txt file
    with open("square_keypoints.txt", "w") as f:
        line = f"{class_id} " + " ".join([f"{kp:.6f}" for kp in keypoints])
        f.write(line)

NameError: name 'i' is not defined

In [1]:
import cv2
for i in range (len(splitImages)):
    
    image = cv2.imread(splitImages[i])
    conors = splitGroundTruth[i]
    points = {k: [int(x), int(y)] for k, v in conors.items() for x, y in [v]}

    # Assign colors to each position (BGR format in OpenCV)
    colors = {
        'top_left': (0, 0, 255),      # Red
        'top_right': (0, 255, 0),     # Green
        'bottom_left': (255, 0, 0),   # Blue
        'bottom_right': (0, 255, 255) # Yellow
    }

    # Create a white image (adjust width/height as needed)
    width, height = 3000, 3000  # Large enough to fit your points

    # Draw the square (connect points with lines)
    cv2.line(image, tuple(points['top_left']), tuple(points['top_right']), (0, 0, 0), 2)
    cv2.line(image, tuple(points['top_left']), tuple(points['bottom_left']), (0, 0, 0), 2)
    cv2.line(image, tuple(points['top_right']), tuple(points['bottom_right']), (0, 0, 0), 2)
    cv2.line(image, tuple(points['bottom_left']), tuple(points['bottom_right']), (0, 0, 0), 2)

    # Draw the points with their respective colors
    for pos, (x, y) in points.items():
        cv2.circle(image, (x, y), 15, colors[pos], -1)  # -1 fills the circle
        cv2.putText(image, pos, (x + 20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    image = cv2.resize(image,(720,720),image)
    # Display the image
    cv2.imshow("Square with Colored Points", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the image (optional)
    cv2.imwrite("square_with_colored_points.jpg", image)

NameError: name 'splitImages' is not defined

In [1]:

import torch
from ultralytics import YOLO

model = YOLO("yolo11n-pose.pt") 
# Train the model
#d:/A_University/ano4/semester_2/Vision/computer-vision/processed/res2k.yaml
model.train(    
    data="./res2k.yaml",
    epochs=100,  # Increased epochs
    imgsz=1024,  # Reduced image size
    batch=8,  # Increased batch size
    workers=2,  # Enable multiprocessing
    name="Yolo11m_v2",
    fliplr=0.0,  # Disable flips
    flipud=0.0,
    degrees=0.0,  # Disable rotation
    perspective=0.0,  # Disable perspective
    optimizer="AdamW",  # More stable optimizer
    lr0=0.001,  # Learning rate
    warmup_epochs=3,  # Gradual learning start
    box=7.5,  # Adjust loss weights
    pose=12.0,
    kobj=1.0,
    overlap_mask=False,)

Ultralytics 8.3.154  Python-3.13.3 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./res2k.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Yolo11m_v25, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=False, patience=100, perspective=0.0, plots=True,

train: Scanning C:\Users\diogo\Desktop\year4\sem2\vc\computer-vision\task3.2_Simple\datasets\2k\train\labels.cache... 1442 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1442/1442 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 4090.61162.1 MB/s, size: 2276.3 KB)


val: Scanning C:\Users\diogo\Desktop\year4\sem2\vc\computer-vision\task3.2_Simple\datasets\2k\val\labels.cache... 330 images, 0 backgrounds, 0 corrupt: 100%|██████████| 330/330 [00:00<?, ?it/s]


Plotting labels to runs\pose\Yolo11m_v25\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 87 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 2 dataloader workers
Logging results to runs\pose\Yolo11m_v25
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        330        330          1          1      0.995      0.995      0.973      0.973      0.965      0.961



100 epochs completed in 1.150 hours.
Optimizer stripped from runs\pose\Yolo11m_v25\weights\last.pt, 5.7MB
Optimizer stripped from runs\pose\Yolo11m_v25\weights\best.pt, 5.7MB

Validating runs\pose\Yolo11m_v25\weights\best.pt...
Ultralytics 8.3.154  Python-3.13.3 torch-2.7.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Laptop GPU, 8151MiB)
YOLO11n-pose summary (fused): 109 layers, 2,654,275 parameters, 0 gradients, 6.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:03<00:00,  5.46it/s]


                   all        330        330          1          1      0.995      0.995      0.994      0.994      0.994       0.98
Speed: 0.4ms preprocess, 4.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs\pose\Yolo11m_v25


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000015885C35B70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.0340

In [ ]:

import torch
from ultralytics import YOLO

model = YOLO("yolo11n-pose.pt") 
# Train the model
#d:/A_University/ano4/semester_2/Vision/computer-vision/processed/res2k.yaml
model.train(    
    data="./res2k.yaml",
    epochs=100,  # Increased epochs
    imgsz=640,  # Reduced image size
    batch=8,  # Increased batch size
    workers=2,  # Enable multiprocessing
    name="Yolo11m_v2",
    fliplr=0.0,  # Disable flips
    flipud=0.0,
    degrees=0.0,  # Disable rotation
    perspective=0.0,  # Disable perspective
    optimizer="AdamW",  # More stable optimizer
    lr0=0.001,  # Learning rate
    warmup_epochs=3,  # Gradual learning start
    box=7.5,  # Adjust loss weights
    pose=12.0,
    kobj=1.0,
    overlap_mask=False,)

New https://pypi.org/project/ultralytics/8.3.154 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.128  Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./res2k.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=Yolo11m_v2, nbs=64, nms=False, opset=None, optim

train: Scanning D:\A_University\ano4\semester_2\Vision\computer-vision\task3.2_Simple\datasets\2k\train\labels.cache... 1442 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1442/1442 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 844.929.3 MB/s, size: 2276.3 KB)


val: Scanning D:\A_University\ano4\semester_2\Vision\computer-vision\task3.2_Simple\datasets\2k\val\labels.cache... 330 images, 0 backgrounds, 0 corrupt: 100%|██████████| 330/330 [00:00<?, ?it/s]


Plotting labels to runs\pose\Yolo11m_v2\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 112 weight(decay=0.0), 122 weight(decay=0.0005), 121 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs\pose\Yolo11m_v2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     46/100      5.32G     0.2611     0.1767          0     0.2155     0.9279          6        640: 100%|██████████| 181/181 [00:58<00:00,  3.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:05<00:00,  3.66it/s]

                   all        330        330          1          1      0.995      0.995          1          1      0.995      0.991



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     47/100      5.32G     0.2654     0.1795          0     0.2115     0.9239         20        640:  11%|█         | 20/181 [00:06<00:50,  3.16it/s]